<a href="https://colab.research.google.com/github/snupet/Twittorials/blob/master/Breadcrumb_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[1] Installs and imports all necessary libraries

In [3]:
%%capture
!pip install --upgrade -q requests-html extruct bs4 lxml pprint plotly==4.9.0 

In [4]:
import pprint
import requests
from extruct.jsonld import JsonLdExtractor
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import lxml
import plotly.express as px
import pandas as pd

[2] Initializes variables

In [5]:
#Holds all URLs with Breadcrumb Lists attributes
allURLs = list()

pp = pprint.PrettyPrinter(indent=1)

user_input = '' #@param {type:"string"}

[3] Function to find breadcrumb trail and add to dictionary

In [6]:
#Finds all URLs with Breadcrumb Lists, counts how many are found, and adds the URL to a list
from collections import defaultdict

def findType(data):
  
  for item in data:
    for k, v in item.items():
      if k == '@type' and v == 'BreadcrumbList':

        itemList = item['itemListElement']

        breadcrumbList = defaultdict(set)

        for breadcrumb in itemList:

          name = breadcrumb["item"]["name"]
          position = breadcrumb["position"]

          breadcrumbList[f"Position{position}"].add(name)


      

        allURLs.append(breadcrumbList)

[4] Iterates through sitemap and finds URLs, then gathers the HTML from the URL, and puts the data into the above function.

In [ ]:
session = HTMLSession()
jslde = JsonLdExtractor()

r = session.get(user_input)
soup = BeautifulSoup(r.text, 'html.parser')
URLs = soup.find_all('loc')
print(len(URLs))

for url in URLs:
  url = url.get_text()
  try:
    html = session.get(url).text
    data = jslde.extract(html)
    findType(data)

  except lxml.etree.ParserError:
    print("No HTML found for the page")
  except requests.exceptions.TooManyRedirects:
    print("Exceeded 30 redirects")
  print(f"Current URL: {url}")

In [ ]:
allURLs

[5] Initializes dataframe and builds treemap from dataframe.

In [11]:
df = pd.DataFrame(allURLs)

In [ ]:
df

In [ ]:
df.info()

If the dataframe above has more than 4 columns, adjust the code below to include more columns. Simply copy the code, change the X in df2[PositionX] = df2.PositionX.astype(str) to the additional column(s). Another line of code must be added per additional column.

In [13]:
from copy import deepcopy
df2 = deepcopy(df)

In [14]:
df2["Position1"] = df2.Position1.astype(str)

In [15]:
df2["Position2"] = df2.Position2.astype(str)

In [16]:
df2["Position3"] = df2.Position3.astype(str)

In [17]:
df2["Position4"] = df2.Position4.astype(str)

Adjust the path=[] to include any additional positions if needed.

In [ ]:
fig = px.treemap(df2, path=['Position1', 'Position2', 'Position3'],
                  color_continuous_scale='RdBu')
fig.show()